# Peripheral showcase: Pi Camera Module 3 in CHI@Edge

Welcome to this Jupyter notebook guide on using the Raspberry Pi Camera Module 3 within CHI@Edge. This artifact will walk you through the steps to access and utilize the camera for your edge computing experiments.. 

**Important Note:** There is only one device with the camera attached at the moment. We currently are in the process of staffing several of our raspberrypi 4 devices with pi camera modules; later, this month of June 2024, an official blogpost on peripherals will accompany this artifact presenting a hollistic approach for users to add peripherals to CHI@Edge.

In the following example we'll show how to use the Pi Camera Module 3 on a Raspberry Pi 4 to capture both a png image and an h264 30 fps video. 

In [ ]:
import chi
chi.use_site("CHI@Edge")

In [15]:
chi.set("project_name", "your_project_name_goes_here")

In [16]:
from chi import container
from chi import lease

## Creating the Lease

To access the camera, we need to make a lease for the specific device that the camera is currently attached to. The device ```sj-rpi4-02``` is specifically set up with the proper kernel and driver options to enable support for the Pi Camera module 3.

In [ ]:
import os

# get your username, just used to name leases something identifiable
username = os.environ.get("USER")

# machine name refers to the "type" of device
machine_name = "raspberrypi4-64"

# Reserving the specific device to which the Pi Camera Module 3 is attached
device_name = "sj-rpi4-02"

# get dates for lease start and end
start, end = lease.lease_duration(days=1)

# make a unique name for the lease
lease_name = f"{username}-{machine_name}-{start}"

reservations = []
lease.add_device_reservation(reservations, count=1, machine_name=machine_name, device_name=device_name)
container_lease = lease.create_lease(lease_name, reservations, start, end)
lease_id = container_lease["id"]

print(f"created lease with name {lease_name} and uuid {lease_id}, waiting for it to start. This can take up to 60s.")
lease.wait_for_active(lease_id)
print("Done!")

## Launching a container with libcamera and the rpicam apps

To take pictures and videos with the camera, we need to launch a container image with all the following pre-requisites:

- pi_camera device profile: a flag to expose all the necessary /dev devices required for the camera support
- libcamera: A complex camera stack designed to support all kinds of cameras on Linux.
- rpicam-apps: A set of example applications demonstrating how to use libcamera on Raspberry Pi.

We compile slightly modified custom versions of both dependencies and package them in our docker image ```ghcr.io/chameleoncloud/edge-picamera-image:latest```. This is currently the only verified image working on CHI@Edge, we provide the [Dockerfile](https://github.com/ChameleonCloud/edge-picamera-image/blob/ea0b0c48fa79c438bf28281bceddd5620fee26ac/Dockerfile) for reproducibility and customization.

In [ ]:
print("Requesting container ... This may take a while as the large (>1GB) container image is being downloaded")

# Set a name for the container. Because CHI@Edge uses Kubernetes, ensure that underscores aren't in the name
container_name = f"tutorial-{machine_name}-picamera".replace("_","-")

try:
    my_container = container.create_container(
        container_name,
        image="ghcr.io/chameleoncloud/edge-picamera-image:latest",
        workdir="/home",
        device_profiles=["pi_camera"],
        reservation_id=lease.get_device_reservation(lease_id),
        platform_version=2,
    )
except RuntimeError as ex:
    print(ex)
    print(f"please stop and/or delete {container_name} and try again")
else:
    print(f"Successfully created container: {container_name}!")

## Capturing a still 1920x1080 picture with the camera

To capture pictures, we use the rpicam applications ([code](https://github.com/raspberrypi/rpicam-apps)). We use ```rpicam-still```[(docs)](https://www.raspberrypi.com/documentation/computers/camera_software.html#rpicam-still) to capture a 1920x1080 still png using the camera. Please visit the rpicam-apps [documentation](https://www.raspberrypi.com/documentation/computers/camera_software.html#rpicam-apps) to learn how to use other utilities.

**Important notes**: 

- Capturing frames can take some time. It is not instant.
- If you run into dma_heap allocation issues/failure to allocate capture buffers, please lower the quality/size of your image or use the png format. The error occurs because the device to which the camera is attached currently only allocates a very limited amount of 64MB of contiguous memory blocks (cma). Each still capture requires a cma buffer amount that is proportional to the size of the target picture. Development efforts are currently ongoing to allow the allocation of more CMA on boot time to allow for larger frames.

In [ ]:
cmd = "rpicam-still -o image.png --width 1920 --height 1080"
print(chi.container.execute(my_container.uuid, cmd)["output"])

Download the captured image from the running container using our ```container.download()``` utility. (**Note**: it takes a slight moment for the jupyter directory navigation pane to refresh and show the downloaded file)

In [37]:
chi.container.download(my_container.uuid, "/home/image.png",".")

## Capturing a 5 second 1080p 60fps H264 video with the camera

To capture an h264 video we use the ```rpicam-vid```[(docs)](https://www.raspberrypi.com/documentation/computers/camera_software.html#rpicam-vid) utility to capture a 5 second 1080p video at a framerate of 60 fps. To playback the video, please download the file through the jupyter interface and use a video playback utility such as ```vlc```.

In [ ]:
cmd = "rpicam-vid -t 5000 --framerate 60 --width 1920 --height 1080 -o video.h264"
print(chi.container.execute(my_container.uuid, cmd)["output"])

**Important note**: the ```container.download()``` utility is primarily meant for smaller size files. Please resort to other methods such as cloud storage or scp to download larger files.

In [38]:
chi.container.download(my_container.uuid, "/home/video.h264",".")